# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import cv2

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


C:\Users\krish\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:20: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.


## Object detection imports
Here are the imports from the object detection module.

In [2]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

In [3]:
# # List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = r'C:\Users\krish\Documents\tensorflow\workspace\training_demo_resnet_2k\annotations\label_map.pbtxt'
PATH_TO_FROZEN_GRAPH = r'C:\Users\krish\Documents\tensorflow\workspace\training_demo_resnet_2k\trained-inference-graphs\output_inference_graph_v1\frozen_inference_graph.pb'

## Load a (frozen) Tensorflow model into memory.

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map

In [5]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Detection

In [6]:
cap = cv2.VideoCapture(r"C:\Users\krish\Desktop\images\freshco_video.mp4")
# Detection
centers = []
c=1
# Detection
with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        while True:
            # Read frame from camera
            ret, image_np = cap.read()
            if c%5==0 and ret == True:
                image_np = cv2.rotate(image_np,cv2.ROTATE_90_CLOCKWISE)
                im_width = image_np.shape[1]
                im_height = image_np.shape[0]
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                # Extract image tensor
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                # Extract detection boxes
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                # Extract detection scores
                scores = detection_graph.get_tensor_by_name('detection_scores:0')
                # Extract detection classes
                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                # Extract number of detectionsd
                num_detections = detection_graph.get_tensor_by_name(
                    'num_detections:0')
                # Actual detection.
                (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                min_score_thresh = 0.5
                for i in range(boxes.shape[1]):
                    if scores is None or scores[0][i] > min_score_thresh:
                        # boxes[i] is the box which will be drawn
                        center_x = int(((boxes[0][i][1]+boxes[0][i][3])/2)*im_width)
                        center_y = int(((boxes[0][i][0]+boxes[0][i][2])/2)*im_height)
                        centers.append([center_x,center_y])
                        

                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image_np,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8)

                # Display output
                cv2.imshow('object detection',image_np)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            elif ret == False:
                break
            c = c+1
        cap.release()
        # Destroy all the windows
        cv2.destroyAllWindows()
                

## Save prediction as video

In [35]:
cap = cv2.VideoCapture(r"C:\Users\krish\Desktop\images\freshco_video.mp4")
out = cv2.VideoWriter(r'C:\Users\krish\Desktop\output1.mp4', -1, 20, (720,1280))
c=1

# Detection
with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        while True:
            # Read frame from camera
            ret, image_np = cap.read()
            if c%5==0 and ret == True:
                image_np = cv2.rotate(image_np,cv2.ROTATE_90_CLOCKWISE)
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                # Extract image tensor
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                # Extract detection boxes
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                # Extract detection scores
                scores = detection_graph.get_tensor_by_name('detection_scores:0')
                # Extract detection classes
                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                # Extract number of detectionsd
                num_detections = detection_graph.get_tensor_by_name(
                    'num_detections:0')
                
                # Actual detection.
                (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                
                min_score_thresh = 0.5
                for i in range(boxes.shape[1]):
                    if scores is None or scores[0][i] > min_score_thresh:
                        # boxes[i] is the box which will be drawn
                        center_x = int(((boxes[0][i][1]+boxes[0][i][3])/2)*im_width)
                        center_y = int(((boxes[0][i][0]+boxes[0][i][2])/2)*im_height)
                        centers.append([center_x,center_y])
                
                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image_np,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8)
                
                
                # Display output
                out.write(image_np)
            
            elif ret == False:
                break
            c = c+1
        cap.release()
        out.release()

## Connect with database and fetch out of stock items

In [10]:
import pymongo

from pymongo import MongoClient

In [11]:
client = MongoClient("localhost", 27017)
db = client["AI_PROJ"]
collect = db["shelfod_video"]

In [41]:
items_out_of_shelf = set(())

for [center_x, center_y] in centers:
    result = collect.find({
        "$and":[
    {"xmin" : { "$lt" : center_x}},
    {"ymin" : { "$lt" : center_y}},
    {"xmax" : { "$gt" : center_x}},
    {"ymax" : { "$gt" : center_y}}
    ]})
    for i in result:
        items_out_of_shelf.add(i["name"])

In [42]:
print(items_out_of_shelf)

{'compliments_berrys', 'silk_almond', 'earths_own_oat_original', 'almond_breeze_orginal', 'earths_own_barista', 'almond_breeze_chocolate', 'zevia_ginger'}
